In [ ]:
#Parte comum:

import pandas as pd
import geopandas as gpd
# from jupyterthemes import jtplot
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.subplots as ps
import basedosdados as bd


states = bd.read_sql(
    '''
    SELECT DISTINCT id_municipio AS id, sigla_uf
    FROM  basedosdados.br_bd_diretorios_brasil.setor_censitario AS censo
    GROUP BY id_municipio, sigla_uf
    ''',
    billing_project_id='geobr-analysis')

cities = gpd.read_file("geojson\\brasil.json", encoding='utf-8')

brasil_data = pd.merge(states, cities, on='id')
brasil_data = gpd.GeoDataFrame(brasil_data, geometry='geometry')

RJ_data = brasil_data[brasil_data["sigla_uf"] == "RJ"].reset_index(drop=True)
RJ_data = RJ_data.astype({"id": int})

covid = pd.read_csv('caso_full.csv.gz', compression = 'gzip')
covid.rename(columns={"city_ibge_code": "id", "last_available_confirmed_per_100k_inhabitants": "cases_per_100k", "last_available_death_rate": "death_rate"}, inplace=True)
covid = covid[covid["id"] > 1000].astype({"id": int})
covid_rj = covid[(covid["state"] == "RJ") & (covid["date"] == "2021-11-17")].reset_index(drop=True)

covid_rj = covid_rj[["id", "estimated_population", "cases_per_100k", "death_rate"]]

RJ_fulldata = pd.merge(RJ_data, covid_rj, on='id')


## PIB per Capita

Avançamos ao terceiro e último parâmetro do IDH. O PIB, Produto Interno Bruto, avalia a soma de todos os bens e serviços finais produzidos, nesse caso, por cada município. Ao fazermos o PIB per Capita, que é o utilizado no cálculo do IDH, dividimos o PIB pela população total de cada cidade do estado. Para tal, utilizamos duas bases do Base dos Dados, uma que continha o PIB e outra com as populações, para efetuarmos esse cálculo:

In [ ]:
pib = bd.read_sql(
    '''
    SELECT id_municipio as id, pib, ano
    FROM `basedosdados.br_ibge_pib.municipio`
    WHERE ano = 2018
    ''',
    billing_project_id='geobr-analysis')

pop = bd.read_sql(
    '''
    SELECT id_municipio as id, populacao as pop
    FROM `basedosdados.br_ibge_populacao.municipio`
    WHERE ano = 2018 AND sigla_uf = 'RJ'
    ''',
    billing_project_id='geobr-analysis')

Com isso, foi possível fazermos as conversões necessárias e o cálculo desejado, além do `merge` com as informações espaciais, possibilitando a plotagem das informações

In [ ]:
pib_data =  pib.astype({"id": int})
pop_data =  pop.astype({"id": int})

pib_data = pd.merge(pib_data, pop_data, on='id')

RJ_pib = pd.merge(RJ_fulldata, pib_data, on='id')

RJ_pib["per_capita"] = RJ_pib["pib"]/RJ_pib["pop"]

RJ_pib.head()

Na escala de cores, usaremos apenas até o valor de 100k, pois caso usássemos o maior valor outlier (200k), o mapa praticamente inteiro ficaria com cores claras demais, o que explicita a desigualdade do PIB entre os municípios. Veja o mapa a seguir:

In [ ]:
fig = px.choropleth(RJ_pib,
                   geojson = RJ_pib["geometry"],
                   locations = RJ_pib.index,
                   color = "per_capita",
                   hover_name = "name",
                   color_continuous_scale = "Greens",
                   range_color=(10000, 100000),
                   labels = {
                       "per_capita": "PIB per Capita"
                   },
)

fig.update_geos(
                projection=dict(
                    scale=55
                    ),
                center = dict(
                    lat=-22.208333,
                    lon=-42.896388
                    )
)

fig.update_layout(
    title_text = "PIB per capita estimado de 2018, por município do RJ",
    margin={"r":0,"t":50,"l":0,"b":50}
)

fig.show()


Certamente a visualização nos dá, intuitivamente, uma resposta. A distribuição é desigual demais para que tenhamos esperança de correlação. Municípios como Maricá e São João da Barra têm PIBs per Capita altíssimos, enquanto Belford Roxo, por exemplo, tem cerca de $10\%$ do valor. Mas façamos a plotagem:

In [ ]:
fig = px.scatter(RJ_pib, x="death_rate", y="per_capita", hover_data=['name'], trendline="ols", 
color_discrete_sequence=['green'],trendline_color_override="orange",
title="Gráfico de dispersão entre taxa de mortalidade e PIB per Capita",
labels={"death_rate": "Taxa de mortalidade",
        "name": "Município",
        "per_capita":"PIB per Capita"})
fig.show()

De fato, não há correlação estabelecida entre os fatores. A distância inicial no gráfico pode dar indicativo de linearidade, mas aproximando a plotagem (o plotly express permite essa função, teste!), vemos a aleatoriedade entre os pontos. O $R^2$ é muito baixo e, de fato, chegamos a uma conclusão final.

## Conclusão

Assim, após analisarmos o próprio PIB e suas três dependências e a relação destes com a pandemia do Covid-19 no estado do Rio de Janeiro, chegamos à conclusao de que não é possível definirmos uma correlação clara entre os fatores, respondendo a pergunta inicial da análise. 

Certamente, a evolução da pandemia teve influência de diversos outros fatores, como densidade demográfica (determinando quais são os grandes enchames urbanos que em geram facilitam a propagação do vírus) ou mesmo aspectos políticos que determinam quais medidas serão tomadas para contenção da evolução do cenário. Certamente, passamos ainda por várias ondas, que foram acompanhadas por guinadas no rumo dos grupos sociais mais diretamente envolvidos em cada fase, etc..
